In [1]:
import os
import json

import numpy as np
import matplotlib.pyplot as plt
import paprika.io as io
from paprika.analysis.utils import get_block_sem, get_nearest_max

from glob import glob
import pytraj as pt

In [2]:
R = 8.314 / 4184
T = 298.15
RT = R*T
gauss_factor = (2*np.pi*np.exp(1))**(3/2)

In [3]:
# Anchor atoms
G1 = ":GST@C10"
G2 = ":GST@C9"
G3 = ":GST@C6"
H1 = ":CB8@C26"
H2 = ":CB8@C2"
H3 = ":CB8@C8"
D1 = ":DM1"
D2 = ":DM2"
D3 = ":DM3"

In [4]:
# Load trajectories
traj = pt.iterload(glob(f"windows/a000/production-v*.dcd"), top="restrained.pdb")

## Translational Entropy

In [5]:
r_guest = pt.center_of_mass(traj, mask=':GST')
r_host = pt.center_of_mass(traj, mask=':CB8')
r_com = r_guest-r_host

In [6]:
covariance = np.cov(r_com.T)
e_val, e_vec = np.linalg.eig(covariance)
translational_volume = gauss_factor*np.prod(np.sqrt(e_val))

In [7]:
dGtrans = -RT*np.log(translational_volume / 1660.5392)

## Rotational Entropy

In [8]:
# Polar angle
theta = pt.angle(traj, mask=f"{D2} {D1} {G1}")*np.pi/180
ave_theta = np.mean(theta)

# Azimuthal angle
phi = pt.dihedral(traj, mask=f"{D1} {G1} {G2} {G3}")*np.pi/180
phi[phi < 0] += 2*np.pi

# Twist angle
psi = pt.dihedral(traj, mask=f"{D2} {D1} {G1} {G2}")*np.pi/180
psi[psi < 0] += 2*np.pi

In [9]:
covariance = np.cov(np.c_[theta, phi, psi].T)
e_val, e_vec = np.linalg.eig(covariance)
rotational_volume = gauss_factor*np.prod(np.sqrt(e_val))*np.sin(ave_theta)

In [10]:
dGrot = -RT*np.log(rotational_volume / (8*np.pi**2))

### Total Entropy

In [11]:
print(f"-TdS(trans) = {dGtrans:.2f} kcal/mol")
print(f"-TdS(rot)   = {dGrot:.2f} kcal/mol")
print(f"-TdS(total) = {dGtrans+dGrot:.2f} kcal/mol")

-TdS(trans) = 3.93 kcal/mol
-TdS(rot)   = 2.48 kcal/mol
-TdS(total) = 6.41 kcal/mol
